In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import importlib.util
import scipy
import scipy.ndimage

# Load rsmf from relative location
spec = importlib.util.spec_from_file_location("rsmf", "../rsmf/__init__.py")
rsmf = importlib.util.module_from_spec(spec)
sys.modules[spec.name] = rsmf
spec.loader.exec_module(rsmf)

## Setup plots

In [2]:
formatter = rsmf.setup("example.tex")

## Simple annotated plot

In [3]:
x = np.linspace(0, 5, 50)
f = (x-1)**2 - x + 0.25 + np.sin(3*x) + 5 * np.exp(-x)
F = (x-1)**3/3. - x**2/2 + 0.25*x + 1/3. - 1/3. * (np.cos(3*x)-1) - 5 * np.exp(-x) + 5

fig = formatter.figure()

plt.plot(x, f)
plt.plot(x, F)

plt.text(4.2,3,'$f(x)$')
plt.text(1.5,7.5, r'$\int_0^x \mathrm{d}\xi \, f(\xi)$')
plt.xlabel('$x$')
plt.tight_layout()
plt.savefig("annotated_plot.pdf")

## More complicated subplot example

In [4]:
# Create a custom cyclic colormap
cdict = {'red':   ((0.0,  0.05, 0.05),
                   (0.25,  1.0, 1.0),
                   (0.5,  0.7, 0.7),
                   (0.75,  0.02, 0.02),
                   (1.0,  0.05, 0.05)),
         'green': ((0.0,  0.27, 0.27),
                   (0.25,  0.72, 0.72),
                   (0.5,  0.02, 0.02),
                   (0.75,  0.80, 0.80),
                   (1.0,  0.27, 0.27)),
         'blue':  ((0.0,  1.0, 1.0),
                   (0.25,  0.02, 0.02),
                   (0.5,  0.21, 0.21),
                   (0.75,  0.12, 0.12),
                   (1.0,  1.0, 1.0))}

my_colormap = mpl.colors.LinearSegmentedColormap('mycolormap', cdict)


# Make plotting reusable
def plot_array(arr, label):
    ax_ = plt.gca()
    X, Y = np.meshgrid(range(arr.shape[0]), range(arr.shape[1]), indexing='ij')
    plt.setp(ax_.get_yticklabels(), visible=False)
    plt.setp(ax_.get_xticklabels(), visible=False)
    for tic in ax_.xaxis.get_major_ticks() + ax_.yaxis.get_major_ticks():
        tic.tick1On = False
        tic.tick2On = False    
    
    im = plt.pcolormesh(X, Y, arr, vmin=0.0, vmax = 2*np.pi, cmap=my_colormap)
    
    ax_.axis([X.min(), X.max(), Y.min(), Y.max()])
    
    # Reduce title fontsize -- it is important to use props here as they guarantee
    # that the fontsize fits well with the surrounding document
    plt.title(label, fontdict = {'fontsize' : formatter.fontsizes.normalsize})
    
    return im

# Create some dummy data
dummy_data = np.random.uniform(0.0, 2 * np.pi, (4,32,32))
dummy_data[1] = scipy.ndimage.uniform_filter(dummy_data[0], size=2, mode='wrap')
dummy_data[2] = scipy.ndimage.uniform_filter(dummy_data[0], size=4, mode='wrap')
dummy_data[3] = scipy.ndimage.uniform_filter(dummy_data[0], size=9, mode='wrap')

# The actual plotting
fig = formatter.figure(aspect_ratio=0.28, wide=True)
    
ax1 = plt.subplot2grid((1,17), (0,0), colspan=4)
ax2 = plt.subplot2grid((1,17), (0,4), colspan=4)
ax3 = plt.subplot2grid((1,17), (0,8), colspan=4)
ax4 = plt.subplot2grid((1,17), (0,12), colspan=4)
ax5 = plt.subplot2grid((1,17), (0,16))

for (ax_, title_, idx_) in [(ax1, '$t = 0$', 0), (ax2, '$t = 1$', 1), (ax3, '$t = 10$', 2), (ax4, '$t = 100$', 3)]:
    plt.sca(ax_)
    im = plot_array(dummy_data[idx_,:,:], title_)      

norm = mpl.colors.Normalize(vmin=0, vmax=2*np.pi)
cb1 = mpl.colorbar.ColorbarBase(ax5, cmap=my_colormap,
                                norm=norm,
                                ticks=[0, np.pi, 2*np.pi],
                                orientation='vertical')
cb1.ax.set_yticklabels(['$0$', '$\\pi$', '$2 \\pi$'])
cb1.set_label('Spin angle')
plt.tight_layout()
plt.savefig("spins.pdf")

## Seaborn integration example

Note that we have to reset the correct fontsizes after applying the seaborn style.

In [5]:
fig = formatter.figure(wide=True)
sns.set(style="ticks")
formatter.set_default_fontsizes()

rs = np.random.RandomState(11)
x = rs.gamma(2, size=1000)
y = -.5 * x + rs.normal(size=1000)

sns.jointplot(x, y, kind="hex", color="#4CB391")

plt.tight_layout()
plt.savefig("hexbin.pdf")